# The Bachelorette Predictor
### Kwame V. Taylor

The goal of this project is to create a machine learning model that accurately predicts how many weeks a contestant will last on the ABC reality show The Bachelorette, based on data from Seasons 1-2 and 4-12.

**Warning: This project contains *many* spoilers for past seasons.** Proceed at your own risk.

<img src="https://miro.medium.com/max/1280/1*S6OS8YFJWx0LT9wXNi0LkQ.png">

## Set up Environment

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
#plt.style.use('ggplot')
import matplotlib as mpl

from cycler import cycler
import seaborn as sns

# default viz size settings
plt.rc('figure', figsize=(12, 8))
plt.rc('font', size=15)
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['lines.linestyle'] = '--'
mpl.rcParams['axes.prop_cycle'] = cycler(color=['deepskyblue', 'firebrick', 'darkseagreen', 'violet'])

import warnings
warnings.filterwarnings("ignore")

In [2]:
from wrangle import acquire_data, join_dfs, drop_extra_cols
from preprocessing import handle_dates_and_elims, train_validate_test

## Acquire & Prepare (Wrangle) the Data

Import raw datasets.

In [17]:
df, join = acquire_data()

In [4]:
df.head(3)

,Name,Age,Occupation,Hometown,ElimWeek,Season
0,Ryan Sutter,29,Firefighter,"Vail, Colorado",NaN,1
1,Charlie Maher,28,Financial Analyst,"Hermosa Beach, California",6.0,1
2,Russ,30,Writer,"San Rafael, California",5.0,1
3,Greg T.,28,Importer,"Manhattan, New York",4.0,1
4,Bob Guiney,31,Mortgage Broker,"Ferndale, Michigan",3.0,1


In [19]:
join.sort_values(by='SEASON').head(3)

,SEASON,CONTESTANT,ELIMINATION-1,ELIMINATION-2,ELIMINATION-3,ELIMINATION-4,ELIMINATION-5,ELIMINATION-6,ELIMINATION-7,ELIMINATION-8,...,DATES-1,DATES-2,DATES-3,DATES-4,DATES-5,DATES-6,DATES-7,DATES-8,DATES-9,DATES-10
281,1,01_WAYNE_X,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,1,01_RYAN_S,NaN,NaN,NaN,NaN,NaN,W,NaN,NaN,...,NaN,D14,D1,D1,D1,D1,NaN,NaN,NaN,NaN
258,1,01_CHARLIE_M,NaN,NaN,NaN,NaN,NaN,E,NaN,NaN,...,NaN,D14,D1,D1,D1,D1,NaN,NaN,NaN,NaN


How to read the ```join``` DataFrame:

Feature         | Description
----------------|------------
`ELIMINATION-1` | Who was eliminated in week 1
`ELIMINATION-2` | Who was eliminated in week 2
`ELIMINATION-3` | Who was eliminated in week 3
`ELIMINATION-4` | Who was eliminated in week 4
`ELIMINATION-5` | Who was eliminated in week 5
`ELIMINATION-6` | Who was eliminated in week 6
`ELIMINATION-7` | Who was eliminated in week 7
`ELIMINATION-8` | Who was eliminated in week 8
`ELIMINATION-9` | Who was eliminated in week 9
`ELIMINATION-10`| Who was eliminated in week 10
`DATES-1`       | Who was on which date in week 1
`DATES-2`       | Who was on which date in week 2
`DATES-3`       | Who was on which date in week 3
`DATES-4`       | Who was on which date in week 4
`DATES-5`       | Who was on which date in week 5
`DATES-6`       | Who was on which date in week 6
`DATES-7`       | Who was on which date in week 7
`DATES-8`       | Who was on which date in week 8
`DATES-9`       | Who was on which date in week 9
`DATES-10`      | Who was on which date in week 10

- Eliminates connote either an elimination (starts with "E") or a rose (starts with "R").
- Eliminations supercede roses.
- "E" connotes a standard elimination, typically at a rose ceremony. "EQ" means the contestant quits. "EF" means the contestant was fired by production. "ED" connotes a date elimination. "EU" connotes an unscheduled elimination, one that takes place at a time outside of a date or rose ceremony.
- "R" means the contestant received a rose. "R1" means the contestant got a first impression rose.
- "D1" means a one-on-one date, "D2" means a 2-on-1, "D3" means a 3-on-1 group date, and so on.
- Weeks of the show are deliminated by rose ceremonies, and may not line up exactly with episodes.

Now I'll join the two dataframes.

In [6]:
df = join_dfs(df, join)

In [7]:
df.head(3)

,Name,Age,Occupation,Hometown,ElimWeek,Season,ELIMINATION-1,ELIMINATION-2,ELIMINATION-3,ELIMINATION-4,...,DATES-1,DATES-2,DATES-3,DATES-4,DATES-5,DATES-6,DATES-7,DATES-8,DATES-9,DATES-10
CONTESTANT,,,,,,,,,,,,,,,,,,,,,
1_RYAN_S,Ryan Sutter,29,Firefighter,"Vail, Colorado",NaN,1,NaN,NaN,NaN,NaN,...,NaN,D14,D1,D1,D1,D1,NaN,NaN,NaN,NaN
1_CHARLIE_M,Charlie Maher,28,Financial Analyst,"Hermosa Beach, California",6.0,1,NaN,NaN,NaN,NaN,...,NaN,D14,D1,D1,D1,D1,NaN,NaN,NaN,NaN
1_RUSS_X,Russ,30,Writer,"San Rafael, California",5.0,1,NaN,NaN,NaN,NaN,...,NaN,D14,D1,D1,D1,NaN,NaN,NaN,NaN,NaN


Drop redundant features and unnecessary features for the project MVP (minimally viable product):
* ```ELIMINATION-1```
* ```ELIMINATION-2```
* ```ELIMINATION-3```
* ```ELIMINATION-4```
* ```ELIMINATION-5```
* ```ELIMINATION-6```
* ```ELIMINATION-7```
* ```ELIMINATION-8```
* ```ELIMINATION-9```
* ```ELIMINATION-10```
* ```Hometown```
* ```Occupation```
* ```Name```
* ```Season```
* ```DATES-1```

In [8]:
df = drop_extra_cols(df)

In [9]:
df.head(3)

,Age,ElimWeek,Season,DATES-2,DATES-3,DATES-4,DATES-5,DATES-6,DATES-7,DATES-8,DATES-9,DATES-10
CONTESTANT,,,,,,,,,,,,
1_RYAN_S,29,NaN,1,D14,D1,D1,D1,D1,NaN,NaN,NaN,NaN
1_CHARLIE_M,28,6.0,1,D14,D1,D1,D1,D1,NaN,NaN,NaN,NaN
1_RUSS_X,30,5.0,1,D14,D1,D1,D1,NaN,NaN,NaN,NaN,NaN


In [10]:
df.isna().sum()

Age           0
ElimWeek      9
Season        0
DATES-2     109
DATES-3     135
DATES-4     159
DATES-5     174
DATES-6     198
DATES-7     215
DATES-8     232
DATES-9     244
DATES-10    253
dtype: int64

## Preprocessing

### Feature Engineering with ```DATES``` and ```ELIMINATION```

* **NaNs in ```ElimWeek``` mean that that contestant won their season. I encoded this as 11.0**, representing that the contestants lasted until/past the end of their season (as Week 10 is the last week).
    * The choice to make winning contests have an ```ElimWeek``` of 11 may cause slightly lower One-on-One scores for winning contests because the dividing value (that I'm using to average the amount of One-on-One time per week) is ```ElimWeek```, yet there are no dates in the 11th week. I will compensate for this in a future iteration.
    * This calculation also does not account for seasons ending early/a contestant getting a win before Week 10. I will address this in a future iteration as well.


* **I dropped ```ELIMINATION``` features** because the presence of values there has a direct relationship with how many weeks a contestant lasts.
    * However, I may engineer features out of these columns, such as ```ReceivedFirstImpressionRose``` that don't have a direct correlation to how many weeks a contestant stays.


* **To quantify the dates as a ```One-on-One_Score```, I made them each a float value** that is equal to ```1``` divided by the ```number of people on the date```.
    * My reasoning behind this is that, anecdotally speaking, the fewer other people present on a date, the more that that date can affect the standing of a contestant on the date.
    * Therefore, the highest date value a contestant can have is if the value is 1, meaning they went on a date in that episode, and they went on that date as a One-on-One (a highly coveted type of date in the world of The Bachelorette).
    
    
* **I created ```First_Date``` feature**.
    * This feature is a boolean that represents if a contestant was chosen to go on a date in the second week of their season, which is the week in which the first dates between the Bachelorette and the contestants occur.
 
 
* **I dropped unneeded columns** that were used to feature-engineer.
 
    
* I want to create a ```Homestate``` feature in a future iteration of this project.

In [11]:
df = handle_dates_and_elims(df)

In [12]:
# quick peek at the data, sorted by age to avoid seeing too much of one season
# and to make sure FirstDate is showing up correctly
df.sort_values(by='Age').head(3)

,Age,ElimWeek,Season,One-on-One_Score,FirstDate
CONTESTANT,,,,,
4_PAUL_B,23,3.0,4,0.060714,1.0
2_CORY_H,24,1.0,2,0.000000,0.0
1_MIKE_X,24,3.0,1,0.067857,1.0


### Train, validate, test splits

```ElimWeek``` is the feature we will be predicting on.

In [13]:
X_train, y_train, X_validate, y_validate, X_test, y_test, train, validate, test = train_validate_test(df, 'ElimWeek')

Shape of train: (147, 4) | Shape of validate: (64, 4) | Shape of test: (53, 4)


In [14]:
X_train.head(3)

,Age,Season,One-on-One_Score,FirstDate
CONTESTANT,,,,
2_RYAN_M,30,2,0.178571,1.0
5_ADAM_D,27,5,0.000000,0.0
4_GRAHAM_B,29,4,0.359524,1.0


In [15]:
y_train.head(3)

CONTESTANT
2_RYAN_M      4.0
5_ADAM_D      1.0
4_GRAHAM_B    6.0
Name: ElimWeek, dtype: float64

### Scale the data

I'll scale the data on my next iteration through the data science pipeline.

## Exploratory Data Analysis

Continue in ```explore.ipynb```. And move old ```explore.ipynb``` to the ```kaggle-datasets/``` folder.

## Modeling

## Conclusions